In [1]:
import os
import sys
import torchvision
from transformers.tokenization_bert import BertTokenizer

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

I1203 02:08:12.486058 139688212862784 file_utils.py:39] PyTorch version 1.1.0 available.


'/work'

In [2]:
sys.path.append(w_dir+'/fgc_support_retri')

In [3]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
from transformers.tokenization_bert import BertTokenizer
import config
from torch.utils.data import DataLoader
from sup_model import BertSupSentClassification
from transformers import BertModel
from tqdm import tqdm_notebook as tqdm
import torch
import torch.nn as nn
import torchvision
from transformers.tokenization_bert import BertTokenizer
import config
from fgc_preprocess import SerDataset, BertIdx, bert_collate
from torch.utils.data import DataLoader
from sup_model import BertSupSentClassification
from transformers import BertModel
from tqdm import tqdm
import ujson
import json
import config
import numpy as np
import torch
from tqdm import tqdm_notebook as tqdm
from torch.utils.data import Dataset
from stanfordcorenlp import StanfordCoreNLP
from utils import read_fgc

I1203 02:08:13.676477 139688212862784 corenlp.py:42] Using an existing server http://140.109.19.191:9000
I1203 02:08:14.679685 139688212862784 corenlp.py:118] The server is available.


In [4]:
class SER_extract:
    def __init__(self):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        bert_model_name = 'bert-base-chinese'
        bert_encoder = BertModel.from_pretrained(bert_model_name)
        bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name)
        bert_indexer = BertIdx(bert_tokenizer)
        model = BertSupSentClassification(bert_encoder)
#         model_path = config.TRAINED_MODELS / '20191129-with_hotpot'/ 'model_epoch5_loss_0.226.m'
        model_path = config.TRAINED_MODELS / '20191128'/ 'model_epoch5_loss_0.213.m' 
        model.load_state_dict(torch.load(model_path, map_location=device))
        model.to(device)
        model.eval()
        
        self.tokenizer = tokenizer = BertTokenizer.from_pretrained(bert_model_name)
        self.model = model
        self.bert_indexer = bert_indexer
        self.device = device
        
    def predict(self, context_sents, question):
        batch = []
        for sent in context_sents:
            sample = self.bert_indexer({'QTEXT':question, 'sentence': sent})
            batch.append(sample)
        batch = bert_collate(batch)      
        with torch.no_grad():
            input_ids = batch['input_ids'].to(self.device)
            token_type_ids = batch['token_type_ids'].to(self.device)
            attention_mask = batch['attention_mask'].to(self.device)
            logits = self.model(input_ids=input_ids, 
                                token_type_ids=token_type_ids,
                                attention_mask=attention_mask, 
                                mode=BertSupSentClassification.ForwardMode.EVAL)
        return logits

In [5]:
ser_extracter = SER_extract()

I1203 02:08:18.577152 139688212862784 configuration_utils.py:152] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json from cache at /root/.cache/torch/transformers/8a3b1cfe5da58286e12a0f5d7d182b8d6eca88c08e26c332ee3817548cf7e60a.0c16faba8be66db3f02805c912e4cf94d3c9cffc1f12fa1a39906f9270f76d33
I1203 02:08:18.579970 139688212862784 configuration_utils.py:169] Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "po

In [6]:
fgc_test_items = read_fgc(config.FGC_DEV)

no gold supporting evidence
{'QID': 'D009Q03', 'QTYPE': '申论', 'QTEXT': '「占领华尔街」运动的诉求为何?', 'ANSWER': [{'ATEXT': '', 'ATOKEN': [{'text': '', 'start': 0}]}], 'ASPAN': [], 'SHINT': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'ATYPE': 'Object', 'AMODE': 'Single-Span-Extraction'}
no gold supporting evidence
{'QID': 'D032Q10', 'QTYPE': '进阶题', 'QTEXT': '第二次签订的北美贸易协定从签署至生效过了几日?', 'ANSWER': [{'ATEXT': '资讯不足无法判定', 'ATOKEN': [{'text': '资讯不足无法判定', 'start': -1}]}], 'ASPAN': [], 'SHINT': [0, 0, 0], 'ATYPE': 'Date-Duration', 'AMODE': 'Date-Duration'}
no gold supporting evidence
{'QID': 'D049Q04', 'QTYPE': '申论', 'QTEXT': '「雅婷逐字稿」的命名起源为何?', 'ANSWER': [{'ATEXT': '', 'ATOKEN': [{'text': '', 'start': 0}]}], 'ASPAN': [], 'SHINT': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'ATYPE': 'Event', 'AMODE': 'Single-Span-Extraction'}
no gold supporting evidence
{'QID': 'D117Q05', 'QTYPE': '进阶题', 'QTEXT': '是否发现肿瘤就是得到癌症?', 'ANSWER': [{'ATEXT': '否', 'ATOKEN': [{'text': '否', 'st

In [7]:
fgc_test_items[0]

{'QID': 'D004Q01',
 'SENTS': ['元祐元年（1086年），',
  '宋哲宗即位，',
  '高太皇太后垂帘听政，',
  '回朝任礼部郎中、中书舍人、翰林学士，',
  '元祐四年（1089年）拜龙图阁学士，',
  '曾出任杭州、颍州等知州职务，',
  '官至礼部尚书。',
  '绍圣元年（1094年）被哲宗贬谪至惠州、儋州（海南岛）。',
  '元符三年（1100年），',
  '宋徽宗即位，',
  '向太后垂帘听政，',
  '下诏让苏轼北还。',
  '建中靖国元年（1101年），',
  '夏天因冷饮过度，',
  '下痢不止，又误服黄芪，',
  '结果病情恶化，',
  '「齿间出血如蚯蚓者无数」，',
  '七月二十八日于常州孙氏馆病卒，',
  '享年六十四岁。',
  '由弟苏辙归葬于郏县小峨眉山。',
  '南宋宋孝宗追赠谥号「文忠」。',
  '苏轼疲于应付新旧党争，',
  '遇事「如食内有蝇，吐之乃已」，',
  '苏轼既反对王安石比较急进的改革措施，',
  '也不同意旧党司马光尽废新法，',
  '所以虽然新党一直称苏轼为旧党，',
  '但其实他在新旧两党之间均受排斥，',
  '仕途坎坷，时常远贬外方，',
  '不过他在各地居官清正，',
  '为民兴利除弊，',
  '政绩颇善，口碑甚佳，',
  '杭州西湖的苏堤就是实证。'],
 'SUP_EVIDENCE': [0,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'QTEXT': '苏东坡在宋哲宗时期,曾出任过哪些官职?',
 'ANS': '礼部郎中、中书舍人、翰林学士、龙图阁学士、知州与礼部尚书'}

In [8]:
tp = 0
gol_t = 0
pre_t = 0

for item in tqdm(fgc_test_items):
    logits = ser_extracter.predict(item['SENTS'], item['QTEXT'])
    logits_sigmoid = torch.sigmoid(logits)
    logits = logits_sigmoid.squeeze(1).cpu().numpy()
    item['logits'] = logits
    prediction = logits_sigmoid > 0.5
    prediction = prediction.squeeze(1).cpu().numpy()
    score_tuple = [(idx,label) for idx, label in enumerate(logits)]
    score_tuple.sort(key=lambda score: score[1], reverse=True)
    topn = int(len(item['SENTS'])/3*1)
    top_scores = score_tuple[:topn]
#     for score in top_scores:
#         prediction[score[0]] = 1
    item['score'] = score_tuple
    
    gold = np.array(item['SUP_EVIDENCE'])

    gol_t += np.count_nonzero(gold == 1)
    pre_t +=  np.count_nonzero(prediction == 1)
    
    if len(gold) != len(prediction):
        print(gold)
        print(prediction)
        print(len(item['SENTS']))
        print(item['SENTS'])
        continue

    for i, gs in enumerate(gold):
        if gs == prediction[i] == 1:
            tp += 1
    item['prediction'] = prediction

precision = tp / pre_t
recall = tp / gol_t

f1 = 2*precision*recall / (precision+recall)

print("precision = {}".format(precision))
print("recall = {}".format(recall))
print("f1 = {}".format(f1))


precision = 0.5806451612903226
recall = 0.13138686131386862
f1 = 0.2142857142857143


In [9]:
fgc_test_items[0]

{'QID': 'D004Q01',
 'SENTS': ['元祐元年（1086年），',
  '宋哲宗即位，',
  '高太皇太后垂帘听政，',
  '回朝任礼部郎中、中书舍人、翰林学士，',
  '元祐四年（1089年）拜龙图阁学士，',
  '曾出任杭州、颍州等知州职务，',
  '官至礼部尚书。',
  '绍圣元年（1094年）被哲宗贬谪至惠州、儋州（海南岛）。',
  '元符三年（1100年），',
  '宋徽宗即位，',
  '向太后垂帘听政，',
  '下诏让苏轼北还。',
  '建中靖国元年（1101年），',
  '夏天因冷饮过度，',
  '下痢不止，又误服黄芪，',
  '结果病情恶化，',
  '「齿间出血如蚯蚓者无数」，',
  '七月二十八日于常州孙氏馆病卒，',
  '享年六十四岁。',
  '由弟苏辙归葬于郏县小峨眉山。',
  '南宋宋孝宗追赠谥号「文忠」。',
  '苏轼疲于应付新旧党争，',
  '遇事「如食内有蝇，吐之乃已」，',
  '苏轼既反对王安石比较急进的改革措施，',
  '也不同意旧党司马光尽废新法，',
  '所以虽然新党一直称苏轼为旧党，',
  '但其实他在新旧两党之间均受排斥，',
  '仕途坎坷，时常远贬外方，',
  '不过他在各地居官清正，',
  '为民兴利除弊，',
  '政绩颇善，口碑甚佳，',
  '杭州西湖的苏堤就是实证。'],
 'SUP_EVIDENCE': [0,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'QTEXT': '苏东坡在宋哲宗时期,曾出任过哪些官职?',
 'ANS': '礼部郎中、中书舍人、翰林学士、龙图阁学士、知州与礼部尚书',
 'logits': array([0.03955746, 0.0477663 , 0.04372749, 0.03534123, 0.10997105,
        0.04597167, 0.04890798, 0.1597471 , 0.02686086, 0.04